In [ ]:
begin
    using Pkg
    Pkg.activate(".")
    Pkg.instantiate()
end

include("modules/DataIO.jl")
include("modules/Model.jl")

using FASTX,
    .DataIO,
    .Model,
    

  Activating project at `~/Desktop/rrm-genomic-extractor`


In [ ]:
modelCachedFile::String = "$(pwd())/.project_cache/extracted_features.dat"
model::Union{Nothing,Dict{String,Tuple{BitArray,Vector{Tuple{Int,Int,Vector{Float64}}},Vector{String}}}} = DataIO.load_cache(modelCachedFile)

Dict{String, Tuple{BitVector, Vector{Tuple{Int64, Int64, Vector{Float64}}}, Vector{String}}} with 5 entries:
  "Omicron" => ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]…
  "Beta"    => ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]…
  "Gamma"   => ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]…
  "Delta"   => ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]…
  "Alpha"   => ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]…

In [39]:
classes = ["Alpha", "Delta", "Beta", "Gamma", "Omicron"]
inputclassdata = Dict{String, Vector{String}}()

for key in classes
    cache_path = "$(pwd())/.project_cache/$(key)_outmask.dat"
    vardata::Union{Nothing,Tuple{String,Tuple{Vector{UInt16},BitArray},Vector{String}}} = DataIO.load_cache(cache_path)
    inputclassdata[key] = vardata[3]
end


In [40]:
mdelstruct = Model.trainModel(inputclassdata, model)

StackOverflowError: StackOverflowError:

In [ ]:
confMatrix = Dict{String,Tuple{Int,Int}}()

Dict{String, Tuple{Int64, Int64}}()

In [ ]:
for class in classes
    classeqs = Vector{String}()
    for record in open(FASTAReader, "/home/salipe/Desktop/datasets/test_voc/test/$class.fasta")
        seq::String = sequence(String, record)
        push!(classeqs, seq)
    end

    classifications = String[]
    for seq in classeqs
        cl, _, _ = classifySequence(mdelstruct, seq)
        # cl, _, _ = Classification.classifyInput(codeunits(seq), model, nothing)
        push!(classifications, cl)
    end
    confMatrix[class] = (count(x -> x == class, classifications), length(classifications))

end
confMatrix